<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6443993b6e50d78ec85a241af9d3a1d3ed792a51eca76569f4b69fc1b3dc457c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-26 12:44:23
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -19.04 K (-0.13%)
Current PnL: -25.56 L (-16.51%)
CY Booked + Current PnL: -11.27 L (-7.28%)
-------------------
Total profit:  1.54 L
Total loss:  -27.10 L
-------------------
Total Booked + Current PnL: 15.71 L (12.29%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.19 L (63.74%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.58%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.61,-18.74,23.18,0.09,18982.0,-18889.0,81888.0,89.97,41.0,M-SC,2.84,253.0,-1.00,0.58,7.58,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.28,8.80,10.01,19.69,20883.0,16878.0,208620.0,-1.77,55.0,X-MC,2.45,78.0,0.81,1.48,16.20,XY25,NTT,AC
50,MASFIN,398.61,-1.65,-3.29,26.20,22.05,24826.0,-3225.0,94755.0,-16.44,50.0,H-SC,6.82,164.0,-0.13,0.67,37.73,XR,ATH,FINANCE
52,NESTLEIND,1377.00,0.45,11.56,8.82,21.40,26116.0,30678.0,296104.0,3.53,59.0,X-LC,5.60,12.0,1.17,2.10,18.97,XY25,NTT,FMCG
43,ITC,452.00,-0.53,-0.87,11.76,10.78,28015.0,-2097.0,238221.0,-41.15,51.0,X-LC,2.02,5.0,-0.07,1.69,4.32,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,5.12,-15.42,72.32,45.74,148129.0,-37353.0,204824.0,12.78,70.0,H-SC,6.18,174.0,-0.25,1.45,33.22,XY24,NTT,ENGINEERING
60,REPCOHOME,880.0,4.51,-10.02,110.96,89.82,286054.0,-28706.0,257799.0,-55.29,59.0,H-SC,5.55,160.0,-0.10,1.83,36.39,XY24,NTT,FINANCE
42,IRCTC,1269.0,3.97,-14.84,79.59,52.93,183337.0,-40156.0,230352.0,-37.01,66.0,H-MC,6.93,118.0,-0.22,1.64,6.85,XY24,BTT,TRAVEL
57,RAJOOENG,143.1,2.96,-37.56,123.59,39.61,79098.0,-38500.0,64000.0,-57.76,36.0,H-SC,31.79,135.0,-0.49,0.45,2.96,AR,ATH,MISC
28,GREENPANEL,537.0,2.44,-37.36,128.51,43.14,147375.0,-68398.0,114680.0,157.11,43.0,M-SC,11.13,240.0,-0.46,0.81,7.87,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
47,KANSAINER,340.00,-3.06,-21.64,44.80,13.47,94671.0,-58347.0,211320.0,-67.73,54.0,H-SC,6.84,158.0,-0.62,1.50,7.45,XY24,NTT,PAINTS
6,ASIANTILES,137.00,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
37,IEX,219.00,-1.93,-7.40,60.50,48.63,112024.0,-14791.0,185163.0,-38.37,39.0,H-SC,13.41,136.0,-0.13,1.31,3.30,XR,NTT,MISC
22,DIXON,18931.72,-1.69,-17.32,52.48,26.07,84707.0,-33813.0,161408.0,-59.80,29.0,X-MC,13.18,56.0,-0.40,1.15,0.53,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
17,COALINDIA,484.83,0.41,-0.65,20.01,19.23,29911.0,-980.0,149480.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.49,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.18,-2.22,112.64,107.91,162612.0,-3280.0,144364.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.03,7.64,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,0.61,-18.74,23.18,0.09,18982.0,-18889.0,81888.0,89.97,41.0,M-SC,2.84,253.0,-1.00,0.58,7.58,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.33,-19.13,23.67,0.02,33416.0,-33385.0,141174.0,115.40,35.0,M-SC,10.58,234.0,-1.00,1.00,22.80,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-3.06,-21.64,44.80,13.47,94671.0,-58347.0,211320.0,-67.73,54.0,H-SC,6.84,158.0,-0.62,1.50,7.45,XY24,NTT,PAINTS
66,SIS,528.00,2.42,-21.79,55.04,21.26,47797.0,-24192.0,86840.0,2051.81,62.0,H-SC,4.84,166.0,-0.51,0.62,17.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
84,WIPRO,420.0,-0.36,9.55,57.25,72.27,106272.0,16180.0,185628.0,-6.08,65.0,M-LC,4.16,99.0,0.15,1.32,16.31,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.36,9.55,57.25,72.27,106272.0,16180.0,185628.0,-6.08,65.0,M-LC,4.16,99.0,0.15,1.32,16.31,XR,NTT,IT
6,ASIANTILES,137.00,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
25,FINCABLES,1641.55,-0.18,-2.22,112.64,107.91,162612.0,-3280.0,144364.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.03,7.64,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.17,-3.98,115.29,106.72,136536.0,-4908.0,118428.0,-52.54,41.0,H-SC,4.74,139.0,-0.04,0.84,18.47,AR,ATH,MISC
1,ABB,7934.00,0.02,-4.30,52.11,45.57,130467.0,-11251.0,250368.0,-38.46,54.0,H-MC,4.52,120.0,-0.09,1.78,6.53,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.69,-17.32,52.48,26.07,84707.0,-33813.0,161408.0,-59.80,29.0,X-MC,13.18,56.0,-0.40,1.15,0.53,X40N,ATH,IT
34,HONAUT,58357.33,-0.45,-18.76,76.41,43.32,101106.0,-30552.0,132320.0,-30.16,29.0,X-SC,6.29,90.0,-0.30,0.94,1.60,X40N,ATH,ELECTRICAL
9,BAJAJHFL,181.50,0.01,-18.48,90.71,55.47,179909.0,-44952.0,198334.0,-27.82,30.0,X-MC,16.06,64.0,-0.25,1.41,1.62,X40N,ATH,FINANCE
8,AWL,485.00,-0.25,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.25,XY24,NTT,FMCG
20,DABUR,735.00,-0.09,-4.18,49.97,43.70,118778.0,-10370.0,237698.0,-12.17,32.0,X-MC,2.11,72.0,-0.09,1.69,10.24,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-0.66,-1.75,23.30,21.14,52042.0,-3981.0,223358.0,-8.73,52.0,X-MC,1.16,74.0,-0.08,1.59,25.90,XY24,NTT,PAINTS
15,CAMS,950.0,-0.46,-0.86,25.56,24.48,63818.0,-2178.0,249678.0,-81.09,47.0,X-SC,1.62,86.0,-0.03,1.77,22.07,X40N,NTT,MISC
78,UNITDSPR,1644.0,0.42,4.91,15.12,20.77,37140.0,11489.0,245633.0,-4.29,51.0,X-MC,1.88,62.0,0.31,1.74,12.16,X40N,NTT,BREWERIES
43,ITC,452.0,-0.53,-0.87,11.76,10.78,28015.0,-2097.0,238221.0,-41.15,51.0,X-LC,2.02,5.0,-0.07,1.69,4.32,X40,NTT,FMCG
20,DABUR,735.0,-0.09,-4.18,49.97,43.70,118778.0,-10370.0,237698.0,-12.17,32.0,X-MC,2.11,72.0,-0.09,1.69,10.24,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.25,-27.09,125.13,64.15,217101.0,-64451.0,173500.0,-57.19,32.0,X-SC,4.52,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.25,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.25,XY24,NTT,FMCG
22,DIXON,18931.72,-1.69,-17.32,52.48,26.07,84707.0,-33813.0,161408.0,-59.80,29.0,X-MC,13.18,56.0,-0.40,1.15,0.53,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.16,-38.84,121.07,35.20,95272.0,-49978.0,78692.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.94,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.83,-19.86,76.58,41.51,161630.0,-52305.0,211060.0,-4.54,42.0,X-MC,7.98,61.0,-0.32,1.50,1.11,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.57,-27.19,96.19,42.84,45527.0,-17676.0,47330.0,-52.07,54.0,X-SC,33.29,83.0,-0.39,0.34,7.08,XY24,NTT,MISC
59,RELAXO,1176.00,-0.73,-47.88,192.25,52.31,145439.0,-69509.0,75651.0,-44.54,42.0,X-SC,7.22,91.0,-0.48,0.54,1.26,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.16,-38.84,121.07,35.20,95272.0,-49978.0,78692.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.94,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.47,-1.00,25.34,24.08,31846.0,-1276.0,125674.0,-9.02,48.0,X-SC,6.65,89.0,-0.04,0.89,18.41,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.45,-18.76,76.41,43.32,101106.0,-30552.0,132320.0,-30.16,29.0,X-SC,6.29,90.0,-0.30,0.94,1.60,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-1.00,-9.65,33.60,20.71,114823.0,-36503.0,341734.0,-22.33,63.0,X-LC,3.00,1.0,-0.32,2.43,14.19,X40,ATH,IT
41,INFY,1972.00,-0.35,12.84,18.97,34.24,66977.0,40181.0,353069.0,-12.06,65.0,X-LC,6.46,2.0,0.60,2.51,22.19,X40,NTT,IT
75,TMPV,600.00,-0.97,-27.10,68.68,22.98,113108.0,-61209.0,164689.0,-24.30,46.0,X-LC,6.22,3.0,-0.54,1.17,3.58,XY24,NTT,AUTO
81,VBL,671.64,0.87,-2.68,39.40,35.67,121111.0,-8454.0,307388.0,-14.21,56.0,X-LC,3.31,4.0,-0.07,2.18,10.81,X40N,ATH,FMCG
43,ITC,452.00,-0.53,-0.87,11.76,10.78,28015.0,-2097.0,238221.0,-41.15,51.0,X-LC,2.02,5.0,-0.07,1.69,4.32,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.63,-37.00,110.93,32.89,53957.0,-28566.0,48641.0,-700.55,56.0,L-MC,6.90,259.0,-0.53,0.35,33.95,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,-1.65,-3.29,26.20,22.05,24826.0,-3225.0,94755.0,-16.44,50.0,H-SC,6.82,164.0,-0.13,0.67,37.73,XR,ATH,FINANCE
6,ASIANTILES,137.00,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
13,BSOFT,831.70,0.07,-18.47,84.95,50.79,97416.0,-25979.0,114674.0,4.75,68.0,H-SC,9.26,151.0,-0.27,0.81,33.52,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.07,-18.47,84.95,50.79,97416.0,-25979.0,114674.0,4.75,68.0,H-SC,9.26,151.0,-0.27,0.81,33.52,XR,ATH,IT
6,ASIANTILES,137.00,-2.54,0.65,80.50,81.67,76008.0,610.0,94420.0,8505.56,71.0,L-SC,14.11,271.0,0.01,0.67,82.85,XR,NTT,CERAMICS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-1.61,-42.73,204.06,74.15,191212.0,-69901.0,93704.0,-268.43,53.0,H-SC,21.81,149.0,-0.37,0.67,20.11,XR,NTT,CHEMICALS
17,COALINDIA,484.83,0.41,-0.65,20.01,19.23,29911.0,-980.0,149480.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.49,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.42
1,25,44.90
2,50,76.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.03
MC    30.77
LC    24.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.89
X40      23.03
X40N     12.88
XR        9.86
XY25      9.16
AR        8.87
OX40N     7.55
SR        0.99
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.17
H-SC    23.91
X-LC    20.70
M-SC    11.50
X-SC     8.07
H-MC     4.85
L-SC     1.55
M-MC     1.40
M-LC     1.32
H-LC     1.11
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.27,-7.20,41.67
IT,13.01,-14.73,72.42
FINANCE,9.66,-17.06,67.98
MISC,7.21,-27.79,80.06
ELECTRICAL,5.96,-12.02,53.05
PAINTS,5.81,-12.82,29.58
INSURANCE,4.69,-2.69,37.83
PHARMA,4.06,-2.96,35.37
AUTO,2.83,-31.51,77.10


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3133176.0,21
AR,1323323.0,10
XR,1274392.0,13
X40,979727.0,14
X40N,872234.0,9
OX40N,727468.0,10
XY25,350311.0,6
SR,281089.0,2
MH,77417.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3610440.0,25
M-SC,1436500.0,15
X-MC,1423790.0,16
X-LC,810045.0,11
X-SC,782801.0,8
H-MC,396786.0,3
L-SC,242755.0,3
M-LC,106272.0,1
H-LC,77815.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1243766.0      6
           AR         906178.0      5
M-SC       XY24       813733.0      6
H-SC       XR         775529.0      7
X-MC       X40        487910.0      7
           XY24       408271.0      3
           X40N       345096.0      4
H-SC       OX40N      326461.0      4
M-SC       OX40N      320125.0      5
X-LC       X40        313853.0      5
X-SC       X40N       310363.0      3
           XY24       294474.0      3
H-SC       SR         281089.0      2
X-LC       X40N       216775.0      2
H-MC       AR         213449.0      2
X-LC       XY24       189595.0      2
H-MC       XY24       183337.0      1
X-MC       XY25       182513.0      2
X-SC       X40        177964.0      2
M-SC       XR         176761.0      2
L-SC       XR         161873.0      2
M-SC       AR         125881.0      2
M-LC       XR         106272.0      1
X-LC       XY25        89822.0      2
L-SC       OX40N       80882.0      1
H-LC       AR          77815.0      1
H-SC       MH          77417.0      1
L-MC       XR          53957.0      1
M-MC       XY25        48065.0      1
L-LC       XY25        29911.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
